In [76]:
from model import model
from model import *
import data_loader
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Data iteration testing

In [30]:
frames = data_loader.get()
imgs= next(iter(frames))

In [31]:
imgs[0].shape

torch.Size([4, 3, 256, 256])

RDN

In [29]:
c = model.conv_feature_extractor.RDN(3,5,5,3,2)

In [32]:
feats=c(test[0])

Create graph from features

In [7]:
grids = torch.cat([x.unsqueeze(0) for x in torch.meshgrid([torch.linspace(-1, 1, 256) for _ in range(2)])])

In [8]:
batch_coords = grids.unsqueeze(0).repeat(4,*[1]*len(grids.shape))

In [33]:
batch_coords_added = torch.cat([batch_coords,feats],axis=1).shape

In [14]:
# Maps x,y to index in flattened list
flat_indices = np.arange(256*256).reshape((256,256)) 
# Create flattened adjacency list
adj_list = [[flat_indices[x,y],flat_indices[neighbx,neighby]]
                # For (very cell
                for x,y in [(x,y) for x in range(256) for y in range(256)]
                    # Every valid neighbor of current cell
                    for neighbx,neighby in [(x+xd,y+yd) for xd in range(-1,2) for yd in range(-1,2) 
                                   if -1<x+xd<256 and -1<y+yd<256] ]
edge_tensor = torch.tensor(adj_list).t().contiguous()

In [22]:
from torch_geometric.data import Data,Batch

In [20]:
batch_edge_index = Batch.from_data_list([Data(edge_index=edge_index)] * batch_size)

NameError: name 'Data' is not defined

In [43]:
flat_feats = torch.flatten(feats,2,3).permute(0,2,1)

In [44]:
tmp=Data(x=flat_feats,edge_index=edge_tensor)

In [45]:
tmp.x.shape

torch.Size([4, 65536, 5])

In [83]:
from torch_geometric.nn import GraphConv,ASAPooling,dense_diff_pool, SAGPooling,graclus
conv = GraphConv(5, 8, node_dim=1)
pool = ASAPooling(5, .2, node_dim=1)
sag = SAGPooling(5, node_dim=1)

In [72]:
convout = conv(tmp.x,tmp.edge_index)

In [79]:
dense_diff_pool(tmp.x[0],tmp.edge_index )

TypeError: dense_diff_pool() missing 1 required positional argument: 's'

In [85]:
graclus(tmp.edge_index).shape

torch.Size([65536])

In [86]:
tmp.edge_index.shape

torch.Size([2, 586756])

In [90]:
256&

65536